# Objectives
This notebook is a new version, that will work on more logged data. The old one is kept for retrocompatibility reasons.

## Data definition

In [ ]:
#target_dir = { # The new study, with more logging about noise as well as rectification for zerosum magnitude 
    #"No noise static" : "my_results/1936484_static_unnoised",
    #"No noise dynamic" : "my_results/1936488_dynamic_unnoised/",
    #"Gaussian /16 static": "my_results/1936483_static_gaussian_16th",
    #"ZeroSum /16 static" : "my_results/1936485_static_zerosum_16th/",
    #"Gaussian /16 dynamic" : "my_results/1936489_dynamic_gaussian_16th/",
    #"ZeroSum /16 dynamic" : "my_results/1936490_dynamic_zerosum_16th/",
    # "Gaussian /32 static": "my_results/1937280_static_gaussian_32th",
    # "ZeroSum /32 static" : "my_results/1937283_static_zerosum_32th",
    # "Gaussian /32 dynamic" : "my_results/1937284_dynamic_gaussian_32th/",
    # "ZeroSum /32 dynamic" : "my_results/1937285_dynamic_zerosum_32th/"
#}

target_dir = { # The new study, with more logging about noise as well as rectification for zerosum magnitude and niid attacks 
    "No noise static" : "my_results/194202_mia_nonoise_static_niidattacks/",
    "No noise dynamic" : "my_results/1942107_mia_nonoise_dynamic_niidattacks/",

    "Gaussian σ/64 static": "my_results/1943115_static_gaussiannoise64th_niidattacks/",
    "Gaussian σ/64 dynamic" : "my_results/1943116_dynamic_gaussiannoise64th_niidattacks/",
    "ZeroSum σ/64 static": "my_results/1943117_static_zerosum64th_niidattacks/" ,
    "ZeroSum σ/64 dynamic" : "my_results/1943118_dynamic_zerosum64th_niidattacks/",

    "Gaussian σ/32 static": "my_results/1942103_mia_gaussian32th_static_niidattacks/",
    "Gaussian σ/32 dynamic" : "my_results/1942104_mia_gaussian32th_dynamic_niidattacks/",
    "ZeroSum σ/32 static":"my_results/1942105_mia_zerosum32th_static_niidattacks/" ,
    "ZeroSum σ/32 dynamic" : "my_results/1942106_mia_zerosum32th_dynamic_niidattacks/",
    
    "Gaussian σ/16 static" : "my_results/1943162_static_gaussiannoise16th_niidattacks/",
    "Gaussian σ/16 dynamic" : "my_results/1943163_dynamic_gaussiannoise16th_niidattacks/",
    "ZeroSum σ/16 static": "my_results/1943164_static_zerosum16th_niidattacks/",
    "ZeroSum σ/16 dynamic" : "my_results/1943165_dynamic_zerosum16th_niidattacks/",
    

    "Gaussian σ/8 static": "my_results/1942108_mia_gaussian8th_static_niidattacks/",
    "Gaussian σ/8 dynamic": "my_results/1942109_mia_gaussian8th_dynamic_niidattacks/",
    "ZeroSum σ/8 static": "my_results/1942112_mia_zerosum8th_static_niidattacks/",
    "ZeroSum σ/8 dynamic" : "my_results/1942113_mia_zerosum8th_dynamic_niidattacks/",

    "Gaussian σ/4 static": "my_results/1942701_static_gaussiannoise4th_niidattacks/",
    "Gaussian σ/4 dynamic" : "my_results/1942702_dynamic_gaussiannoise4th_niidattacks/",
    "ZeroSum σ/4 static": "my_results/1942703_static_zerosum4th_niidattacks/",
    "ZeroSum σ/4 dynamic" : "my_results/1942704_dynamic_zerosum4th_niidattacks/",

    "Gaussian σ/2 static": "my_results/1942845_static_gaussiannoise2th_niidattacks/",
    "Gaussian σ/2 dynamic" : "my_results/1942843_dynamic_gaussiannoise2th_niidattacks/",
    "ZeroSum σ/2 static": "my_results/1942844_static_zerosum2th_niidattacks/",
    "ZeroSum σ/2 dynamic" : "my_results/1942846_dynamic_zerosum2th_niidattacks/",

}


TOTAL_PROCESSES = 36
MAX_MACHINES =  3
MAX_ITERATIONS=4000
STARTING_ITERATION = 0

linestyles = {
    "ZeroSum" : "--"
}

fontsize=20

alpha = 0.1

save_directory = "assets/"

## Data loading:
Non private data:

In [ ]:
import pandas as pd
import numpy as np
from numpy import nansum
from numpy import nanmean

import os
import matplotlib.pyplot as plt

assert TOTAL_PROCESSES%MAX_MACHINES == 0
MAX_PROCESSES = TOTAL_PROCESSES//MAX_MACHINES

machine_folder = 'machine{}'
result_file = '{}_results.json'


# Load 
data_dict = {}
for key in target_dir:
    data_dict[key] = pd.DataFrame({})
    
    
for (key, dir) in target_dir.items():
    print(f"Loading results for run {key} at folder {dir}.  ")
    for machine in range(MAX_MACHINES):
        for rank in range(MAX_PROCESSES):
            print(f"Loading results for machine {machine} and rank {rank}.  ",end = "\r")
            uid = rank + machine * MAX_PROCESSES

            file = os.path.join(dir, machine_folder.format(machine), result_file.format(rank))
            tmp_df = pd.read_json(file)
            tmp_df["uid"] = uid # Manually add the uid for further processing                                                                   
            tmp_df["iteration"] = tmp_df.index
            # print(tmp_df)
            tmp_df = tmp_df[tmp_df["iteration"]>=STARTING_ITERATION]
            tmp_df = tmp_df[tmp_df["iteration"]<=MAX_ITERATIONS]
            data_dict[key] = pd.concat([data_dict[key],tmp_df])

            
print("Loading finished!" + " "*40)
# data_dict['No noise static']

In [ ]:
def get_iteration_result(data):
    data_collapsed = data.groupby(by="iteration",dropna = False)

    res = data_collapsed.agg({  'train_loss':['mean','std','min','max'], 
                                "test_loss":['mean','std','min','max'], 
                                "test_acc": ['mean','std','min','max'],
                                "test_niid_loss" : ['mean','std','min','max'],
                                "test_niid_acc" : ['mean','std','min','max'],
                                "generated_noise_std" : ["mean",'std','min','max']
                              })
    res.reset_index(inplace=True)
    return res


aggregated_data_label_dict = {}

for label,data in data_dict.items():
    print(f"Handling {label}")
    data = data[["iteration","uid","train_loss","test_loss","test_acc","test_niid_loss","test_niid_acc","generated_noise_std"]] # Since we are interested in losses
    

    #data.dropna(["train_loss","test_loss","test_acc","test_niid_loss","test_niid_acc"],inplace=True)
    
    #data_dict[label] = data
    data_aggregated = get_iteration_result(data)
    aggregated_data_label_dict[label] = data_aggregated
    

# aggregated_data_label_dict

In [ ]:
aggregated_data_label_dict["No noise static"]["iteration"]

Load privacy data as well:


In [ ]:
LOCATIONS_OF_ATTACKS = ["PRE-STEP", "POST-STEP","PRE-STEP-niid","POST-STEP-niid"]



assert TOTAL_PROCESSES%MAX_MACHINES == 0
MAX_PROCESSES = TOTAL_PROCESSES//MAX_MACHINES
# FILE_NAMES = [f"privacy-summary-{loc}.json" for loc in LOCATIONS_OF_ATTACKS]
machine_folder = 'machine{}'
privacy_folder = 'privacy'
summary_folder = 'summary'
process_folder = '{}'

def load_privacy_data(path_dir):
    data = {}

    for loc in LOCATIONS_OF_ATTACKS:
        location = f"privacy-summary-{loc}.json"
        data[loc] = pd.DataFrame({})  
        for machine in range(MAX_MACHINES):
            for rank in range(MAX_PROCESSES):
                print(f"Loading {location} for machine {machine} and rank {rank}  ",end = "\r")
                file = os.path.join(path_dir, machine_folder.format(machine),privacy_folder, summary_folder, process_folder.format(machine*MAX_PROCESSES+rank), location)
                tmp_df = pd.read_json(file)
                tmp_df = tmp_df[tmp_df.iteration <= MAX_ITERATIONS]
                tmp_df = tmp_df[tmp_df.iteration >= STARTING_ITERATION]
                #tmp_df['location_of_attack']= file.split('.')[0]
                data[loc] = pd.concat([data[loc],tmp_df])
    return data

privacy_data_dict = {}
for key,dir in target_dir.items():
    print(f"Loading privacy data for {key} at \"{dir}\"")
    privacy_data_dict[key] = load_privacy_data(dir)

print("Loading finished!" + " "*40)
# data_dict

# Global loss display:

In [ ]:
# Must be user defined, feel free to adjust figsize (limits to the naïve autoscaling performed here)
figsize = (20,50)
alpha = 0.1

attributes = ["train_loss", "test_loss", "test_acc","test_niid_loss","test_niid_acc","generated_noise_std"]
metrics = ["mean"]  # The metric we want to evaluate, must be computed in the table

to_plot = [ 
    "No noise static",
    "No noise dynamic",

    "Gaussian σ/64 static",
    # "Gaussian σ/64 dynamic",

    "Gaussian σ/32 static",
    # "Gaussian σ/32 dynamic",

    "Gaussian σ/16 static",
    # "Gaussian σ/16 dynamic",

    "Gaussian σ/8 static",
    # "Gaussian σ/8 dynamic",

    "Gaussian σ/4 static",
    # "Gaussian σ/4 dynamic",

    "Gaussian σ/2 static",
    # "Gaussian σ/2 dynamic",

    "ZeroSum σ/64 static",
    # "ZeroSum σ/64 dynamic",

    "ZeroSum σ/32 static",
    # "ZeroSum σ/32 dynamic",
    
    "ZeroSum σ/16 static",
    # "ZeroSum σ/16 dynamic",
    
    "ZeroSum σ/8 static",
    # "ZeroSum σ/8 dynamic",
    
    "ZeroSum σ/4 static",
    # "ZeroSum σ/4 dynamic"

    "ZeroSum σ/2 static",
    # "ZeroSum σ/2 dynamic"
]




subplot_dim = (len(attributes),len(metrics))
fig,axs = plt.subplots(subplot_dim[0],subplot_dim[1],sharex= True, figsize=figsize)

if subplot_dim[1] ==1:
    axs = [[ax] for ax in axs]
# Loop to set axis and sublpot titles. 
for i,attribute in enumerate(attributes):
    for j, metric in enumerate(metrics):
        if "test" in attribute:
            axs[i][j].set_title(f"{metric} of {attribute}, evaluated on global test set",fontsize=fontsize)
        elif "train" in attribute: 
            axs[i][j].set_title(f"{metric} of {attribute}, evaluated on local train set",fontsize=fontsize) 
        else:
            axs[i][j].set_title(f"{metric} of {attribute}",fontsize=fontsize)
        axs[i][j].set_xlabel(f"Communication rounds",fontsize=fontsize)
        axs[i][j].set_ylabel(f"{attribute} {metric}",fontsize=fontsize)

#Plots the data
for key in to_plot:
    data = aggregated_data_label_dict[key] 
    for i, attribute in enumerate(attributes):
        for j, metric in enumerate(metrics):
            line = "-"
            for substring,linestyle in linestyles.items():
                if substring in key:
                    line = linestyle
            no_nan_data = data[attribute][metric].dropna()
            axs[i][j].plot(no_nan_data.index,no_nan_data,line,label=key)  
            if metric=="mean": # When displaying the mean, we also display the min and max for each iteration. 
                min_data = data[attribute]["min"].dropna()
                max_data = data[attribute]["max"].dropna()
                axs[i][j].fill_between(min_data.index, min_data, max_data, alpha=alpha)
   

for i in range(subplot_dim[0]):
    for j in range(subplot_dim[1]):
        axs[i][j].legend(ncol=3,fontsize=3*fontsize/4)
        axs[i][j].grid()
        axs[i][j].tick_params(labelbottom=True,labelleft = True,axis="both", labelsize=fontsize) 
        extent = axs[i][j].get_tightbbox(fig.canvas.get_renderer()).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig(f"{save_directory}/{axs[i][j].get_title()}", bbox_inches=extent)

## Privacy attack display:

The relevant data must have been loaded by now, into `privacy_data_dict`

In [ ]:
metrics = ["Attacker advantage","AUC"]
metrics = [metrics[0]]
figsize = (15,15)
LOCATIONS_OF_ATTACKS = ["PRE-STEP", "POST-STEP"] # The one we want to show here
window_size = 10
alpha=0.1
to_plot = [ 
    "No noise static",
    # "No noise dynamic",

    # "Gaussian σ/64 static",
    # "Gaussian σ/64 dynamic",

    # "Gaussian σ/32 static",
    # "Gaussian σ/32 dynamic",

    "Gaussian σ/16 static",
    # "Gaussian σ/16 dynamic",

    "Gaussian σ/8 static",
    # "Gaussian σ/8 dynamic",

    "Gaussian σ/4 static",
    # "Gaussian σ/4 dynamic",

    "Gaussian σ/2 static",
    # "Gaussian σ/2 dynamic",

    # "ZeroSum σ/64 static",
    # "ZeroSum σ/64 dynamic",

    # "ZeroSum σ/32 static",
    # "ZeroSum σ/32 dynamic",
    
    "ZeroSum σ/16 static",
    # "ZeroSum σ/16 dynamic",
    
    "ZeroSum σ/8 static",
    # "ZeroSum σ/8 dynamic",
    
    "ZeroSum σ/4 static",
    # "ZeroSum σ/4 dynamic",

    "ZeroSum σ/2 static",
    # "ZeroSum σ/2 dynamic"
]

# to_plot = [ 
#     "No noise static",
#     "No noise dynamic",

#     # "Gaussian σ/64 static",
#     # "Gaussian σ/64 dynamic",

#     # "Gaussian σ/32 static",
#     # "Gaussian σ/32 dynamic",

#     # "Gaussian σ/16 static",
#     # "Gaussian σ/16 dynamic",

#     # "Gaussian σ/8 static",
#     # "Gaussian σ/8 dynamic",

#     "Gaussian σ/4 static",
#     "Gaussian σ/4 dynamic",

#     "Gaussian σ/2 static",
#     "Gaussian σ/2 dynamic",

#     # "ZeroSum σ/64 static",
#     # "ZeroSum σ/64 dynamic",

#     # "ZeroSum σ/32 static",
#     # "ZeroSum σ/32 dynamic",
    
#     # "ZeroSum σ/16 static",
#     # "ZeroSum σ/16 dynamic",
    
#     # "ZeroSum σ/8 static",
#     # "ZeroSum σ/8 dynamic",
    
#     "ZeroSum σ/4 static",
#     "ZeroSum σ/4 dynamic",

#     "ZeroSum σ/2 static",
#     "ZeroSum σ/2 dynamic"
# ]



linewidth = 3


def plot_privacy_data(to_plot,LOCATIONS_OF_ATTACKS):
    fig,axs = plt.subplots(len(metrics),2,sharey='row',sharex= True, figsize=(figsize[0]*2,figsize[1]*len(metrics)))

    if len(metrics) ==1:
        axs = [axs]
    # Group by iteration and extracted mean and std
    for label in to_plot: 
        data = privacy_data_dict[label]
        line_style = '-'
        for key,line in linestyles.items():
            if key in label:
                line_style =  line
        # entire_dataset = data[data['slice feature'] == 'Entire dataset']
        for i,metric in enumerate(metrics):
            for j,location in enumerate(LOCATIONS_OF_ATTACKS):
                complementary_name = ""
                if "niid" in location:
                    complementary_name = " NIID"
                location_label = "".join(location.split('-'))

                # Extract relevant data
                location_data = data[location][data[location]['slice feature'] == 'Entire dataset']

                # Aggregate
                columns = ['iteration', 'Attacker advantage', 'AUC']
                averaged = location_data[columns].groupby('iteration').agg([np.max, np.min, np.mean, np.std])
                
                # Plot
                metric_data = averaged[metric].rolling(window=window_size).mean()
                            
                data_max, data_min = metric_data["amax"], metric_data["amin"]


                mean = metric_data['mean']
                std = metric_data['std']
                axs[i][j].plot(averaged.index, mean, line_style, label= label + "-MEAN",linewidth=linewidth*2)
                axs[i][j].fill_between(averaged.index, mean-2*std, mean + 2*std, alpha=alpha)
                axs[i][j].set_title(f"Evolution of the {metric} | {location} | Window : {window_size}",fontsize=fontsize)
                axs[i][j].set_ylabel(metric+complementary_name, fontsize=fontsize)
                axs[i][j].set_xlabel('Communication rounds', fontsize=fontsize)
                
    for i,metric in enumerate(metrics):
        for j,location in enumerate(LOCATIONS_OF_ATTACKS):
            axs[i][j].legend(fontsize=fontsize,ncol=2)
            axs[i][j].tick_params(labelbottom=True,labelleft = True)
            axs[i][j].tick_params(axis="both", labelsize=fontsize) 
            axs[i][j].grid()
            extent = axs[i][j].get_tightbbox(fig.canvas.get_renderer()).transformed(fig.dpi_scale_trans.inverted())
            fig.savefig(f"{save_directory}/{axs[i][j].get_title()}", bbox_inches=extent)


    

plot_privacy_data(to_plot,LOCATIONS_OF_ATTACKS)


In [ ]:
LOCATIONS_OF_ATTACKS = ["PRE-STEP-niid","POST-STEP-niid"] # The one we want to show here

plot_privacy_data(to_plot,LOCATIONS_OF_ATTACKS)


## Tentative d'affichage loss/privacy

In [ ]:
attributes = ["train_loss", "test_loss","test_acc","test_niid_loss","test_niid_acc"]
# attributes = ["test_acc","test_niid_acc"]
figsize = (20,5)
point_size = 30

to_plot = [ 
    "No noise static",
    # "No noise dynamic",

    # "Gaussian σ/64 static",
    # "Gaussian σ/64 dynamic",

    "Gaussian σ/32 static",
    # "Gaussian σ/32 dynamic",

    "Gaussian σ/16 static",
    # "Gaussian σ/16 dynamic",

    "Gaussian σ/8 static",
    # "Gaussian σ/8 dynamic",

    "Gaussian σ/4 static",
    # "Gaussian σ/4 dynamic",

    "Gaussian σ/2 static",
    # "Gaussian σ/2 dynamic",

    # "ZeroSum σ/64 static",
    # "ZeroSum σ/64 dynamic",

    "ZeroSum σ/32 static",
    # "ZeroSum σ/32 dynamic",
    
    "ZeroSum σ/16 static",
    # "ZeroSum σ/16 dynamic",
    
    "ZeroSum σ/8 static",
    # "ZeroSum σ/8 dynamic",
    
    "ZeroSum σ/4 static",
    # "ZeroSum σ/4 dynamic"

    "ZeroSum σ/2 static",
    # "ZeroSum σ/2 dynamic"
]

min_iteration = 3000
max_iteration = 4000


# shapes = {
#     ("Gaussian","static") : "s",
#     ("Gaussian","dynamic") : "*",
#     ("ZeroSum","static") : "^",
#     ("ZeroSum","dynamic") : "v",
#     ("No noise static"): "P",
#     ("No noise dynamic") : "X"
# }

# color_groups = {
#     "No noise": "blue",
#     "σ/64": "cyan",
#     "σ/32": "green",
#     "σ/16": "y",
#     "σ/8" :"grey",
#     "σ/4" :"brown",
#     "σ/2" : "red"

# }

shapes = {
    ("No noise",): "o",
    ("σ/64",) : "1",
    ("σ/32",): "*",
    ("σ/16",): "D",
    ("σ/8",): "X",
    ("σ/4",): "^",
    ("σ/2",): "s"
}

color_groups = {
    ("Gaussian","static"): "blue",
    ("Gaussian","dynamic") : "cyan",
    ("ZeroSum","static"): "red",
    ("ZeroSum","dynamic"): "orange",
    ("No noise static"):"darkgreen",
    ("No noise dynamic") :"limegreen",
}


locations = ["PRE-STEP", "POST-STEP", "PRE-STEP-niid", "POST-STEP-niid"]
# locations = ["PRE-STEP"]


from matplotlib.patches import Patch
from matplotlib.lines import Line2D

def clip_iterations_bounds(data,min_iteration,max_iteration):
    if "iteration" in data.columns:
        tmp_df= data[data["iteration"] <= max_iteration]
        tmp_df = tmp_df[tmp_df["iteration"] >= min_iteration]

        return tmp_df 
    else:

        print(data.columns)
        print(data)
        raise KeyError(f"`iteration` key not in the dataframe column, probably missing a reset_index")

def get_scatterplot(utility_data,privacy_data,attribute,min_iteration,max_iteration):

    data = clip_iterations_bounds(utility_data, min_iteration=min_iteration, max_iteration=max_iteration)
    data.reset_index(inplace=True)
    data = data[data[attribute]["mean"].isnull() == False]
    x_axis = pd.concat([data["iteration"],data[attribute]["mean"]],axis=1, join="inner")
    # print(x_axis)
   
    privacy_dataset = privacy_data[privacy_data['slice feature'] == 'Entire dataset']
    privacy_dataset = clip_iterations_bounds(privacy_dataset,min_iteration=min_iteration, max_iteration=max_iteration)

    # location_data = privacy_dataset[privacy_dataset.location_of_attack==location]

    # Aggregate the privacy data
    columns = ['iteration', 'Attacker advantage', 'AUC']
    averaged = privacy_dataset[columns].groupby('iteration').agg(["mean"])
    averaged.reset_index(inplace=True)

    # TODO: This is a quick fix because of inconsistency logging/attack rate, the last iteration is not attacked.
    # Should be fixed in the future, and this removed / a sanity check should be added
    if len(x_axis) != len(averaged['Attacker advantage']):
        x_axis = x_axis[:-1:]

    # print(f"{key} {location} : {len(x_axis)}, {len(averaged['Attacker advantage'])}              ", end = "\r")

    assert x_axis["iteration"].reindex(x_axis["iteration"]).index.equals(averaged["iteration"].reindex(averaged["iteration"]).index),\
            f"Got different iterations : {x_axis['iteration'].reindex(x_axis['iteration']).index} and {averaged['iteration'].reindex(averaged['iteration']).index}."

    return x_axis["mean"], averaged

def find_discriminator(key,shapes):
    # print(shapes)
    if shapes is not None:
        for shape_discriminator,shape in shapes.items():
            all_attributes_in = True
            for attribute in shape_discriminator: #All the attributes of the discriminator must be matched in the name we want to display. 
                if attribute not in key:
                    all_attributes_in = False
                    break
            if all_attributes_in:
                # print(f"{key} matched by {shape_discriminator}")
                return shape_discriminator
    # print(f"RETURNED NONE on {key}")
    return None

def format_attribute(attr):
    if type(attr) != tuple:
        return attr
    for i,attribute in enumerate(attr):
        if i != 0:
            formatted += " + " + attribute
        else:
            formatted = attribute
    return formatted

def get_custom_legend(shapes,color_groups):
    custom_legend = []
    for label, shape in shapes.items():
        formatted_label = format_attribute(label)
        custom_legend.append(Line2D([0], [0], marker=shape, linewidth=0,color='blue', label=formatted_label, markersize=15))
    
    for label,color in color_groups.items():
        formatted_label = format_attribute(label)
        custom_legend.append(Line2D([0], [0], marker=".", color=color, lw=4, label=formatted_label))
    return custom_legend

def plot_privacyvsloss(to_plot, locations = ["PRE-STEP", "POST-STEP"], display_type = "all" ,shapes = None,\
                        min_iteration=STARTING_ITERATION, max_iteration=MAX_ITERATIONS):
    subplot_dim = (len(attributes), len(locations))
    fig,axs = plt.subplots(subplot_dim[0], subplot_dim[1],sharey='row',sharex= False, figsize=(figsize[1]* subplot_dim[0], figsize[0] * subplot_dim[1]))


    for i,attribute in enumerate(attributes):
        for j,location in enumerate(locations):
            legend_elements=None
            if display_type=="center":
                legend_elements = get_custom_legend(shapes,color_groups)
           
            axs[i][j].set_xlabel(attribute,fontsize=fontsize*2/3)
            suffix = "IID"
            if "niid" in location: 
                suffix = "NIID"
            axs[i][j].set_ylabel(f"Attacker advantage {suffix}",fontsize=fontsize*2/3)
            if display_type == "all":
                axs[i][j].set_title(f"Attacker advantage vs {attribute} at {location}, iterations {min_iteration}-{max_iteration}",fontsize=fontsize*2/3)
            elif display_type == "center":
                points_history = {}
                for shape_key in color_groups.keys():
                    points_history[shape_key] = []
                axs[i][j].set_title(f"Centroid of attacker advantage vs {attribute} at {location}, iterations {min_iteration}-{max_iteration}",fontsize=fontsize*2/3)
            for key in to_plot:
                data = aggregated_data_label_dict[key]
                privacy_data = privacy_data_dict[key][location]
                
                x_axis, averaged = get_scatterplot(data,privacy_data,attribute,min_iteration,max_iteration)

                shape_discriminator = find_discriminator(key,shapes)
                marker = None
                if shape_discriminator is not None:
                    marker = shapes[shape_discriminator]

                color_discriminator = find_discriminator(key,color_groups)
                color = color_groups[color_discriminator]
                # print(f"{key} : {color_discriminator}, {color}")
                # for color_attribute,c in color_groups.items():
                #     if color_name in key:
                #         color = c
                #         break

                if display_type == "all":
                    axs[i][j].scatter(x_axis,averaged["Attacker advantage"], s= point_size,marker = marker, color=color, label = key)
                elif display_type == "center":
                    # axs[i][j].scatter(x_axis,averaged["Attacker advantage"], s= point_size,marker = marker, color='w', alpha=0)
                    axs[i][j].scatter(x_axis,averaged["Attacker advantage"], s= point_size,marker = marker,color = color,alpha=0)
                    x_value = np.mean(x_axis)
                    y_value = averaged["Attacker advantage"].mean(axis=0)
                    if color_discriminator is not None:
                        points_history[color_discriminator].append((x_value,y_value))

                    axs[i][j].scatter(x_value,y_value, s= point_size * 10,marker = marker, color = color, label = key)

          
            if display_type == "center":
                for key, coordinates in points_history.items():
                    if len(coordinates)>1: #To discard the unnoised baselines.
                        formatted_label = format_attribute(key)
                        p1, = axs[i][j].plot([x[0] for x in coordinates],[x[1] for x in coordinates],label = formatted_label,color=color_groups[key])
                        # legend_elements.append(p1)
    
            axs[i][j].legend(fontsize=fontsize*2/3,ncol=3,handles=legend_elements)

    for ax_list in axs:
        for ax in ax_list:
            ax.tick_params(labelbottom=True,labelleft = True,axis="both", labelsize=2*fontsize/3)
            ax.grid()
            extent = ax.get_tightbbox(fig.canvas.get_renderer()).transformed(fig.dpi_scale_trans.inverted())
            fig.savefig(f"{save_directory}/{ax.get_title()}", bbox_inches=extent)

plot_privacyvsloss(to_plot,["PRE-STEP","POST-STEP"], shapes = shapes, min_iteration=min_iteration, max_iteration=max_iteration)

### Attacker advantage on NIID testset:

In [ ]:
plot_privacyvsloss(to_plot,["PRE-STEP-niid","POST-STEP-niid"], shapes = shapes, min_iteration=min_iteration, max_iteration=max_iteration)

### Affichage des centres de gravité


In [ ]:
to_plot = [ 
    "No noise static",
    # "No noise dynamic",

    "Gaussian σ/64 static",
    # "Gaussian σ/64 dynamic",

    "Gaussian σ/32 static",
    # "Gaussian σ/32 dynamic",

    "Gaussian σ/16 static",
    # "Gaussian σ/16 dynamic",

    "Gaussian σ/8 static",
    # "Gaussian σ/8 dynamic",

    "Gaussian σ/4 static",
    # "Gaussian σ/4 dynamic",

    "Gaussian σ/2 static",
    # "Gaussian σ/2 dynamic",

    "ZeroSum σ/64 static",
    # "ZeroSum σ/64 dynamic",

    "ZeroSum σ/32 static",
    # "ZeroSum σ/32 dynamic",
    
    "ZeroSum σ/16 static",
    # "ZeroSum σ/16 dynamic",
    
    "ZeroSum σ/8 static",
    # "ZeroSum σ/8 dynamic",
    
    "ZeroSum σ/4 static",
    # "ZeroSum σ/4 dynamic"

    "ZeroSum σ/2 static",
    # "ZeroSum σ/2 dynamic"
]

to_plot = [ 
    "No noise static",
    "No noise dynamic",

    "Gaussian σ/64 static",
    "Gaussian σ/64 dynamic",

    "Gaussian σ/32 static",
    "Gaussian σ/32 dynamic",

    "Gaussian σ/16 static",
    "Gaussian σ/16 dynamic",

    "Gaussian σ/8 static",
    "Gaussian σ/8 dynamic",

    "Gaussian σ/4 static",
    "Gaussian σ/4 dynamic",

    "Gaussian σ/2 static",
    "Gaussian σ/2 dynamic",

    "ZeroSum σ/64 static",
    "ZeroSum σ/64 dynamic",

    "ZeroSum σ/32 static",
    "ZeroSum σ/32 dynamic",
    
    "ZeroSum σ/16 static",
    "ZeroSum σ/16 dynamic",
    
    "ZeroSum σ/8 static",
    "ZeroSum σ/8 dynamic",
    
    "ZeroSum σ/4 static",
    "ZeroSum σ/4 dynamic",

    "ZeroSum σ/2 static",
    "ZeroSum σ/2 dynamic"
]

shapes = {
    ("No noise",): "o",
    ("σ/64",) : "1",
    ("σ/32",): "*",
    ("σ/16",): "D",
    ("σ/8",): "X",
    ("σ/4",): "^",
    ("σ/2",): "s"
}

color_groups = {
    ("Gaussian","static"): "blue",
    ("Gaussian","dynamic") : "cyan",
    ("ZeroSum","static"): "red",
    ("ZeroSum","dynamic"): "orange",
    ("No noise static"):"darkgreen",
    ("No noise dynamic") :"limegreen",
}

plot_privacyvsloss(to_plot,["PRE-STEP","POST-STEP"],display_type = "center", shapes = shapes, min_iteration=min_iteration, max_iteration=max_iteration)

In [ ]:
plot_privacyvsloss(to_plot,["PRE-STEP-niid","POST-STEP-niid"],display_type = "center", shapes = shapes, min_iteration=min_iteration, max_iteration=max_iteration)

## Utility comparison display
We will simply present the utility of each method in a clearer manner here.

In [ ]:
for key in aggregated_data_label_dict.keys():
    print(f"\"{key}\",")

def get_key_list(attributes_lists):
    potential = target_dir.keys()
    for attribute in attributes_lists:
        potential = [key for key in potential if attribute in key]
    return potential


In [ ]:
metrics = ["test_loss","test_acc","test_niid_acc","train_loss"]
figsize = (20,4)
alpha = 0.4
width = 0.25  # the width of the bars
window_size = 10
ncols = 1

topology = "static"


prop_cycle = plt.rcParams['axes.prop_cycle']
total_colors = prop_cycle.by_key()['color']

attributes = [ #The attributes to display and color by
    (["Gaussian", "static"],total_colors[0]),
    # (["Gaussian", "dynamic"],total_colors[1]),
    (["ZeroSum", "static"],total_colors[2]),
    # (["ZeroSum", "dynamic"],total_colors[3]) 
]


baselines = {
    f"No noise {topology}" : "b"
}

to_plot = [["σ/64"],["σ/32"], ["σ/16"], ["σ/8"], ["σ/4"], ["σ/2"]]
for e in to_plot:
    e.append(topology)



to_plot_dict = {}
for attr_list in to_plot:
    to_plot_dict[attr_list[0]] = get_key_list(attributes_lists=attr_list)


def is_attribute_match(name,attribute_list):
    is_match = True
    for attribute in attribute_list:
        if attribute not in name:
            return False
    return True

def get_last_n_indexes(data_to_consider,window_size):
    data = data_to_consider.tail(window_size)
    return data.index

def plot_utility(metric,to_plot_dict,baselines, window_size):
    fig,ax = plt.subplots(figsize=figsize)
    x = np.arange(len(to_plot_dict.keys()))
    indexes = None
    multiplier = 0
    total_plot = {}
    for name,keys in to_plot_dict.items():
        for key in keys:
            data = aggregated_data_label_dict[key]
            data_to_consider = data[metric]["mean"].dropna()
            if indexes is None:
                indexes = get_last_n_indexes(data_to_consider, window_size)
                # print(indexes)
            data_to_plot = np.mean(data_to_consider[indexes])
            # print(f"{key} : {data_to_plot}")
            for attribute_list,color in attributes:
                
                if is_attribute_match(key,attribute_list):
                    attribute = " ".join(attribute_list)
                    if (attribute,color) not in total_plot.keys():
                        total_plot[(attribute,color)] = []
                    total_plot[(attribute,color)].append(data_to_plot)

    for (attribute,color), data_to_plot in total_plot.items():
        offset = width * multiplier
        rects = plt.bar(x + offset, data_to_plot, width, label=attribute,color=color)
        plt.bar_label(rects, padding=len(keys))
        multiplier += 1

                # y.append(data_to_plot)

    #Draw the baseline:
    for key,color in baselines.items():
        data = aggregated_data_label_dict[key]
        data_to_consider = data[metric]["mean"].dropna()
        data_to_plot = np.mean(data_to_consider[indexes])
        plt.axhline(y=data_to_plot,linewidth=1,label = key,linestyle ="--",color= color)

    # plt.bar(x,y,color = total_colors,alpha = alpha)
    ax.set_xticks(x + ((len(keys)-1) * width/2), to_plot_dict.keys(),fontsize=fontsize)
    plt.ylabel(f"{metric}",fontsize=fontsize)
    plt.grid()
    if topology != None:
        plt.title(f"Mean of last {window_size} {metric} comparison, {topology} topology")
    else:
        plt.title(f"Mean of last {window_size} {metric} comparison")
    plt.legend(fontsize=fontsize*2/3,ncol = ncols )
    plt.tick_params(labelbottom=True,labelleft = True,axis="both", labelsize=2*fontsize/3)
    extent = fig.axes[0].get_tightbbox(fig.canvas.get_renderer()).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(f"{save_directory}/{fig.axes[0].get_title()}", bbox_inches=extent)
    # plt.rc('ytick',labelsize=fontsize)
    plt.show()


for name,keys in to_plot_dict.items():
    print(f"{name} : {keys}")
for metric in metrics:
    plot_utility(metric,to_plot_dict,baselines,window_size)


    

In [ ]:
ncols=1

attributes = [ #The attributes to display and color by
    # (["Gaussian", "static"],total_colors[0]),
    (["Gaussian", "dynamic"],total_colors[1]),
    # (["ZeroSum", "static"],total_colors[2]),
    (["ZeroSum", "dynamic"],total_colors[3]) 
]
topology="dynamic"
baselines = {
    f"No noise {topology}" : "r"
}
to_plot = [["σ/64"],["σ/32"], ["σ/16"], ["σ/8"], ["σ/4"], ["σ/2"]]
for e in to_plot:
    e.append(topology)

to_plot_dict = {}
for attr_list in to_plot:
    to_plot_dict[attr_list[0]] = get_key_list(attributes_lists=attr_list)


for name,keys in to_plot_dict.items():
    print(f"{name} : {keys}")
for metric in metrics:
    plot_utility(metric,to_plot_dict,baselines,window_size)


In [ ]:
width = 0.20  # the width of the bars
ncols=2
attributes = [ #The attributes to display and color by
    (["Gaussian", "static"],total_colors[0]),
    (["Gaussian", "dynamic"],total_colors[1]),
    (["ZeroSum", "static"],total_colors[2]),
    (["ZeroSum", "dynamic"],total_colors[3]) 
]
topology = None

baselines = {
    f"No noise static" : "b",
    f"No noise dynamic" : "r"
}
to_plot = [["σ/64"],["σ/32"], ["σ/16"], ["σ/8"], ["σ/4"], ["σ/2"]]



to_plot_dict = {}
for attr_list in to_plot:
    to_plot_dict[attr_list[0]] = get_key_list(attributes_lists=attr_list)


for name,keys in to_plot_dict.items():
    print(f"{name} : {keys}")
for metric in metrics:
    plot_utility(metric,to_plot_dict,baselines,window_size)

In [ ]:
to_plot = {
    "Gaussian static" : [key for key in target_dir.keys() if "Gaussian" in key and "static" in key],
    "Gaussian dynamic" : [key for key in target_dir.keys() if "Gaussian" in key and "dynamic" in key],
    "ZeroSum static" : [key for key in target_dir.keys() if "ZeroSum" in key and "static" in key],
    "ZeroSum dynamic" : [key for key in target_dir.keys() if "ZeroSum" in key and "dynamic" in key],
    "No noise static" : ["No noise static"],
    "No noise dynamic" : ["No noise dynamic"]
}

baseline = ["No noise static", "No noise dynamic"]
colors = {
    "static" : "b",
    "dynamic" : "r"
}
metrics = ["test_acc","test_niid_acc", "train_loss"]

for metric in metrics:
    fig = plt.figure(figsize = figsize)
    indexes = None
    for expe_type, expe_list in to_plot.items():
        
        if len(expe_list) == 1 and expe_list[0] in baseline:
            expe = expe_list[0] 
            data = aggregated_data_label_dict[expe][metric]["mean"].dropna()
            
            if indexes is None:
                indexes = get_last_n_indexes(data,window_size)
                print(indexes)
            y_value = np.mean(data[indexes])
            found = False
            for substring,color in colors.items():
                if substring in expe:
                    plt.axhline(y=y_value,linewidth=1, color=color,label = expe,linestyle ="--")
                    found = True
                    break
            if not found:
                plt.axhline(y=y_value,linewidth=1,label = expe,linestyle ="--")
        else:
            data_to_plot = []
            for expe in expe_list:
                data = aggregated_data_label_dict[expe]
                x_value = np.mean(data["generated_noise_std"]["mean"])

                data = data[metric]["mean"].dropna()
                if indexes is None:    
                    indexes = get_last_n_indexes(data,window_size)
                    print(indexes)
                y_value = np.mean(data[indexes])
                data_to_plot.append((x_value,y_value))

            data_to_plot.sort(key = lambda x : x[0])

            plt.plot([x for (x,y) in data_to_plot],[y for (x,y) in data_to_plot], marker = 'o', label=expe_type)

    
    plt.legend(fontsize=fontsize*2/3)
    plt.xlabel("Noise level",fontsize=fontsize)
    plt.ylabel(metric,fontsize=fontsize)
    plt.tick_params(labelbottom=True,labelleft = True,axis="both", labelsize=2*fontsize/3)
    plt.title(f"Evolution of the last {window_size} {metric} according to noise level",fontsize=fontsize)
    plt.grid()
    ax = fig.axes[0]
    extent = ax.get_tightbbox(fig.canvas.get_renderer()).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(f"{save_directory}/{ax.get_title()}", bbox_inches=extent)
    plt.show()